In [44]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## 🧠 **User Query**

In [45]:
query = "Is it charge any international charges?"

### VecDB and Retriever Part same as previous parts 

In [46]:
from langchain.vectorstores import faiss
from langchain.embeddings import AzureOpenAIEmbeddings

In [47]:
embeddings = AzureOpenAIEmbeddings(
	azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    	azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    	openai_api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
    	model="text-embedding-3-small",
	chunk_size=1
    )

In [48]:
faiss_index = faiss.FAISS.load_local("my_faiss_index", embeddings ,allow_dangerous_deserialization=True)

In [49]:
retriever = faiss_index.as_retriever(search_kwargs={"k": 3})

## 🧱 **Context Assembly**

This step combines the relevant retrieved chunks into a single prompt context.

In [50]:
contexts = retriever.get_relevant_documents(query)
context = "\n\n".join([doc.page_content for doc in contexts])

## 📝 **Prompt Formatting**

The context and user query are formatted into a structured prompt for the language model.

In [51]:
from langchain.prompts import ChatPromptTemplate

In [52]:
prompt = """
You are a customer service agent for a mobile phone company. You have been given the following information about the customer query and the context.
Customer Query: {query}
Context: {context}

Answer: 
Your task is to answer the customer query based on the context provided.
Please answer in a friendly and professional manner.
"""

In [ ]:
final_prompt = prompt.format(context=context, query=query)
print(final_prompt[:500]+"...")


You are a customer service agent for a mobile phone company. You have been given the following information about the customer query and the context.
Customer Query: Is it charge any international charges?
Context: and may be restricted to the original country of sale. Call 
charges and international shipping charges may apply, 
depending on the location. Subject to the full terms and 
detailed information on obtaining service available at 
apple.com/legal/warranty  and support.apple.com , if yo


## 🤖 **Language Model Input**

The final prompt is passed to the LLM for answer generation.

In [54]:
from langchain.chat_models import AzureChatOpenAI

In [55]:
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_CHAT_API_VERSION"],
)

## 🧠 **Response Generation**

The language model returns an answer based on the given context.

In [56]:
from langchain.schema import HumanMessage
response = llm([HumanMessage(content=final_prompt)])

/var/folders/39/wh_jffd90vv2lczbpfngvr200000gn/T/ipykernel_40419/3024218951.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=final_prompt)])


In [57]:
print(response.content)

Thank you for reaching out! Regarding your question about international charges, it’s important to note that call charges and international shipping charges may apply depending on your location. If you’re looking to make calls or use services that incur international fees, those charges will vary based on your specific plan and destination. 

For the most accurate information, I recommend checking your mobile plan details or contacting your service provider directly. If you have any more questions or need further assistance, feel free to ask!


## 📤 **Answer Output (Post-processing)**

The final answer is displayed to the user. It can be returned in chat, UI, or API response.

In [58]:
answer = response.content
print("\n✅ Final Answer:\n", answer)


✅ Final Answer:
 Thank you for reaching out! Regarding your question about international charges, it’s important to note that call charges and international shipping charges may apply depending on your location. If you’re looking to make calls or use services that incur international fees, those charges will vary based on your specific plan and destination. 

For the most accurate information, I recommend checking your mobile plan details or contacting your service provider directly. If you have any more questions or need further assistance, feel free to ask!


<strong><h1 style="text-align: center">End of Notebook</h1></strong></div>